# Urban Water Model Simulation for Fehraltorf

This notebook runs the urban water model simulation for Fehraltorf and visualizes the results.

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd

import holoviews as hv
from bokeh.io import output_notebook
hv.extension('bokeh')
output_notebook()

from duwcm import run
from duwcm.functions import load_config
from duwcm.data_structures import UrbanWaterData

from duwcm.viz import (
    plot_aggregated_results,
    create_map_base,
    create_dynamic_map,
    create_flow_visualization
)

## Configuration

Set up the configuration for the simulation.

In [ ]:
config_path = 'config.toml'
config = load_config(config_path, env='default')

print(f"Input directory: {config.input_directory}")
print(f"Output directory: {config.output.output_directory}")
print(f"Simulation period: {config.simulation.start_year} - {config.simulation.end_year}")

geo_file = Path(config.input_directory) / Path(config.files.geo_file)
background_file = Path(config.geodata_directory) / config.files.background_shapefile

base_map = create_map_base(geo_file, background_file)
base_map.show()

## Run Simulation

Execute the urban water model simulation.

In [ ]:
results, model_params, forcing_data, flow_paths = run(config)

## Visualize Results

Create plots to visualize the simulation results.

In [ ]:
fig = plot_aggregated_results(results['aggregated'], forcing_data)
fig.show()

## Generate Interactive Maps

Create interactive map visualizations of the results.

In [ ]:
area_variables = [
    'evapotranspiration',
    'imported_water',
    'baseflow',
    'deep_seepage',
    'stormwater_runoff',
    'wastewater_discharge'
]

gdf_geometry = gpd.read_file(geo_file)
time_series_data = results['local'][area_variables].unstack(level='cell')
time_series_data.index = pd.to_datetime(time_series_data.index)

dynamic_map = create_dynamic_map(gdf_geometry, area_variables, time_series_data)
dynamic_map.show()

## Generate Alluvial diagram

In [ ]:
sankey_fig = create_flow_visualization(results, viz_type='sankey')
sankey_fig.show()

## Generate Chord diagram

In [ ]:
chord_fig = create_flow_visualization(results, viz_type='chord') 
chord_fig